# 基于 GraphSage 的无监督学习


GraphScope提供了处理学习任务的功能。本次教程，我们将会展示GraphScope如何使用GraphSage算法训练一个无监督学习模型。

本次教程的学习任务是链接预测，通过计算在图中顶点之间存在边的概率来预测链接。

在这一任务中，我们使用GraphScope内置的GraphSage算法在[PPI](https://humgenomics.biomedcentral.com/articles/10.1186/1479-7364-3-3-291)数据集上训练一个模型，这一训练模型可以用来预测蛋白质结构之间的链接。这一任务可以被看作在一个异构链接网络上的无监督训练任务。

在这一任务中，GraphSage算法会将图中的结构信息和属性信息压缩为每个节点上的低维嵌入向量，这些嵌入和表征可以进一步用来预测节点间的链接。

这一教程将会分为以下几个步骤：
- 建立会话和载图
- 启动GraphScope的学习引擎，并将图关联到引擎上
- 使用内置的GCN模型定义训练过程，并定义相关的超参
- 开始训练


首先，我们要新建一个会话，并载入数据

In [ ]:
import os
import graphscope

k8s_volumes = {
    "data": {
        "type": "hostPath",
        "field": {
          "path": "/testingdata",
          "type": "Directory"
        },
        "mounts": {
          "mountPath": "/home/jovyan/datasets",
            "readOnly": True
        }
    }
}

# 建立会话
graphscope.set_option(show_log=True)
sess = graphscope.session(show_log=True, k8s_volumes=k8s_volumes)

# 加载PPI图数据
graph = sess.load_from(
    edges={
        "link": [
            (
                "/home/jovyan/datasets/ppi/edge.csv",
                [],
                ("src_id", "protein"),
                ("dst_id", "protein"),
            ),
        ]
    },
    vertices={
        "protein": (
            "/home/jovyan/datasets/ppi/node.csv",
        ),
    },
)

## Launch learning engine

然后，我们需要定义一个特征列表用于图的训练。训练特征集合必须从点的属性集合中选取。在这个例子中，我们选择了属性集合中所有以"feat-"为前缀的属性作为训练特征集，这一特征集也是PPI数据中点的特征集。

借助定义的特征列表，接下来，我们使用会话的`learning`方法来开启一个学习引擎。（`learning`方法的文档可参考[Session](https://graphscope.io/docs/reference/session.html))

在这个例子中，我们在`learning`方法中，指定在数据中`protein`类型的顶点和`link`类型边上进行模型训练。

利用`gen_labels`参数，我们将`protein`点数据集作为训练集。

In [ ]:
# define the features for learning
paper_features = []
for i in range(50):
    paper_features.append("feat-" + str(i))

# launch a learning engine.
lg = sess.learning(graph, nodes=[("protein", paper_features)],
                  edges=[("protein", "link", "protein")],
                  gen_labels=[
                      ("train", "protein", 100, (0, 100)),
                  ])



这里我们使用内置的GraphSage模型定义训练过程。你可以在[Graph Learning Model](https://graphscope.io/docs/learning_engine.html#data-model)获取更多内置学习模型的信息。


在本次示例中，我们使用tensorflow作为NN后端训练器。

In [ ]:

import numpy as np
from graphscope.learning.examples import GraphSage
from graphscope.learning.graphlearn.python.model.tf.trainer import LocalTFTrainer
from graphscope.learning.graphlearn.python.model.tf.optimizer import get_tf_optimizer

# unsupervised GraphSage.

def train(config, graph):
    def model_fn():
        return GraphSage(
            graph,
            config["class_num"],
            config["features_num"],
            config["batch_size"],
            categorical_attrs_desc=config["categorical_attrs_desc"],
            hidden_dim=config["hidden_dim"],
            in_drop_rate=config["in_drop_rate"],
            neighs_num=config["neighs_num"],
            hops_num=config["hops_num"],
            node_type=config["node_type"],
            edge_type=config["edge_type"],
            full_graph_mode=config["full_graph_mode"],
            unsupervised=config['unsupervised'],
        )
    trainer = LocalTFTrainer(
        model_fn,
        epoch=config["epoch"],
        optimizer=get_tf_optimizer(
            config["learning_algo"], config["learning_rate"], config["weight_decay"]
        ),
    )
    trainer.train()
    embs = trainer.get_node_embedding()
    np.save(config['emb_save_dir'], embs)

# define hyperparameters
config = {
    "class_num": 128,  # output dimension
    "features_num": 50,
    "batch_size": 512,
    "categorical_attrs_desc": "",
    "hidden_dim": 128,
    "in_drop_rate": 0.5,
    "hops_num": 2,
    "neighs_num": [5, 5],
    "full_graph_mode": False,
    "agg_type": "gcn",  # mean, sum
    "learning_algo": "adam",
    "learning_rate": 0.01,
    "weight_decay": 0.0005,
    'unsupervised': True,
    "epoch": 1,
    'emb_save_dir': './id_emb',
    "node_type": "protein",
    "edge_type": "link",
}

## Run training process


在定义完训练过程和超参后，现在我们可以使用学习引擎和定义的超参开始训练过程。

In [ ]:
train(config, lg)


训练完毕后，需要关掉会话

In [ ]:
sess.close()